http://machinelearningmastery.com/understanding-stateful-lstm-recurrent-neural-networks-python-keras/

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import keras
import string

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.utils import np_utils

In [21]:
np.random.seed(7)

In [27]:
alphabet = string.ascii_uppercase
len(alphabet)

26

In [28]:
# define the raw dataset
alphabet = string.ascii_uppercase
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))


In [158]:
# 1. takes text or any input data = alphabet
# 2. separates it into two lists: dataX = list with elements of length seq_length 
# 3. dataY = with elements that is immediately next to the last element of seq_length

# prepare the dataset of input to output pairs encoded as integers
seq_length = 1

dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]

    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    print(seq_in, '->', seq_out)

A -> B
B -> C
C -> D
D -> E
E -> F
F -> G
G -> H
H -> I
I -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> Q
Q -> R
R -> S
S -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z


In [159]:
dataX

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24]]

In [160]:
"""We need to reshape the NumPy array into a format expected by the LSTM networks,
that is [samples, time steps, features]."""

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))
X

array([[[ 0]],

       [[ 1]],

       [[ 2]],

       [[ 3]],

       [[ 4]],

       [[ 5]],

       [[ 6]],

       [[ 7]],

       [[ 8]],

       [[ 9]],

       [[10]],

       [[11]],

       [[12]],

       [[13]],

       [[14]],

       [[15]],

       [[16]],

       [[17]],

       [[18]],

       [[19]],

       [[20]],

       [[21]],

       [[22]],

       [[23]],

       [[24]]])

In [161]:
"""Once reshaped, we can then normalize the input integers to the range 0-to-1,
the range of the sigmoid activation functions used by the LSTM network.
"""
X = X / float(len(alphabet))
X

array([[[ 0.        ]],

       [[ 0.03846154]],

       [[ 0.07692308]],

       [[ 0.11538462]],

       [[ 0.15384615]],

       [[ 0.19230769]],

       [[ 0.23076923]],

       [[ 0.26923077]],

       [[ 0.30769231]],

       [[ 0.34615385]],

       [[ 0.38461538]],

       [[ 0.42307692]],

       [[ 0.46153846]],

       [[ 0.5       ]],

       [[ 0.53846154]],

       [[ 0.57692308]],

       [[ 0.61538462]],

       [[ 0.65384615]],

       [[ 0.69230769]],

       [[ 0.73076923]],

       [[ 0.76923077]],

       [[ 0.80769231]],

       [[ 0.84615385]],

       [[ 0.88461538]],

       [[ 0.92307692]]])

In [162]:
"""Finally, we can think of this problem as a sequence classification task, 
where each of the 26 letters represents a different class. As such, we can convert the output (y) 
to a one hot encoding, using the Keras built-in function to_categorical().
"""
# one hot encode the output variable
y = keras.utils.to_categorical(dataY)
y

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [163]:
char_to_int["B"]

1

In [217]:
"""
Let’s define an LSTM network with 32 units and a single output neuron with a softmax activation function 
for making predictions. Because this is a multi-class classification problem, 
we can use the log loss function (called “categorical_crossentropy” in Keras), 
and optimize the network using the ADAM optimization function.
The model is fit over 500 epochs with a batch size of 1.
"""

# create and fit the model
model = keras.models.Sequential()
model.add(keras.layers.LSTM(32, input_shape=(X.shape[1], X.shape[2])))
model.add(keras.layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=1000, batch_size=1, verbose=1)

Epoch 1/1000
25/25 [==============================] - 1s - loss: 3.2650 - acc: 0.0000e+00      
Epoch 2/1000
25/25 [==============================] - 0s - loss: 3.2576 - acc: 0.0400         
Epoch 3/1000
25/25 [==============================] - 0s - loss: 3.2550 - acc: 0.0400     
Epoch 4/1000
25/25 [==============================] - 0s - loss: 3.2522 - acc: 0.0800         
Epoch 5/1000
25/25 [==============================] - 0s - loss: 3.2499 - acc: 0.0800         
Epoch 6/1000
25/25 [==============================] - 0s - loss: 3.2469 - acc: 0.0400         
Epoch 7/1000
25/25 [==============================] - 0s - loss: 3.2440 - acc: 0.0400         
Epoch 8/1000
25/25 [==============================] - 0s - loss: 3.2418 - acc: 0.0400         
Epoch 9/1000
25/25 [==============================] - 0s - loss: 3.2386 - acc: 0.0400         
Epoch 10/1000
25/25 [==============================] - 0s - loss: 3.2355 - acc: 0.0400         
Epoch 11/1000
25/25 [==============================]

25/25 [==============================] - 0s - loss: 2.5019 - acc: 0.1200         
Epoch 87/1000
25/25 [==============================] - 0s - loss: 2.4942 - acc: 0.1200     
Epoch 88/1000
25/25 [==============================] - 0s - loss: 2.4890 - acc: 0.1200         
Epoch 89/1000
25/25 [==============================] - 0s - loss: 2.4844 - acc: 0.1200         
Epoch 90/1000
25/25 [==============================] - 0s - loss: 2.4768 - acc: 0.1200         
Epoch 91/1000
25/25 [==============================] - 0s - loss: 2.4703 - acc: 0.1600         
Epoch 92/1000
25/25 [==============================] - 0s - loss: 2.4668 - acc: 0.1600         
Epoch 93/1000
25/25 [==============================] - 0s - loss: 2.4598 - acc: 0.1200         
Epoch 94/1000
25/25 [==============================] - 0s - loss: 2.4543 - acc: 0.1600         
Epoch 95/1000
25/25 [==============================] - 0s - loss: 2.4497 - acc: 0.1600         
Epoch 96/1000
25/25 [==============================] - 0s 

25/25 [==============================] - 0s - loss: 2.1782 - acc: 0.2800     
Epoch 172/1000
25/25 [==============================] - 0s - loss: 2.1766 - acc: 0.3200         
Epoch 173/1000
25/25 [==============================] - 0s - loss: 2.1745 - acc: 0.3200         
Epoch 174/1000
25/25 [==============================] - 0s - loss: 2.1727 - acc: 0.2800         
Epoch 175/1000
25/25 [==============================] - 0s - loss: 2.1688 - acc: 0.2800         
Epoch 176/1000
25/25 [==============================] - 0s - loss: 2.1667 - acc: 0.3600         
Epoch 177/1000
25/25 [==============================] - 0s - loss: 2.1644 - acc: 0.3600     
Epoch 178/1000
25/25 [==============================] - 0s - loss: 2.1614 - acc: 0.2800         
Epoch 179/1000
25/25 [==============================] - 0s - loss: 2.1605 - acc: 0.4000         
Epoch 180/1000
25/25 [==============================] - 0s - loss: 2.1587 - acc: 0.3600         
Epoch 181/1000
25/25 [==============================]

25/25 [==============================] - 0s - loss: 1.9998 - acc: 0.4400     
Epoch 258/1000
25/25 [==============================] - 0s - loss: 2.0001 - acc: 0.3600     
Epoch 259/1000
25/25 [==============================] - 0s - loss: 1.9964 - acc: 0.6000         
Epoch 260/1000
25/25 [==============================] - 0s - loss: 1.9959 - acc: 0.5200         
Epoch 261/1000
25/25 [==============================] - 0s - loss: 1.9919 - acc: 0.4800         
Epoch 262/1000
25/25 [==============================] - 0s - loss: 1.9915 - acc: 0.4800     
Epoch 263/1000
25/25 [==============================] - 0s - loss: 1.9900 - acc: 0.4400     
Epoch 264/1000
25/25 [==============================] - 0s - loss: 1.9869 - acc: 0.5600     
Epoch 265/1000
25/25 [==============================] - 0s - loss: 1.9861 - acc: 0.4400     
Epoch 266/1000
25/25 [==============================] - 0s - loss: 1.9846 - acc: 0.5600     
Epoch 267/1000
25/25 [==============================] - 0s - loss: 1.9834

25/25 [==============================] - 0s - loss: 1.8729 - acc: 0.6000         
Epoch 344/1000
25/25 [==============================] - 0s - loss: 1.8688 - acc: 0.6800     
Epoch 345/1000
25/25 [==============================] - 0s - loss: 1.8683 - acc: 0.6400     
Epoch 346/1000
25/25 [==============================] - 0s - loss: 1.8657 - acc: 0.6800     
Epoch 347/1000
25/25 [==============================] - 0s - loss: 1.8651 - acc: 0.6400     
Epoch 348/1000
25/25 [==============================] - 0s - loss: 1.8647 - acc: 0.6800         
Epoch 349/1000
25/25 [==============================] - 0s - loss: 1.8615 - acc: 0.6000     
Epoch 350/1000
25/25 [==============================] - 0s - loss: 1.8579 - acc: 0.6400     
Epoch 351/1000
25/25 [==============================] - 0s - loss: 1.8611 - acc: 0.6000     
Epoch 352/1000
25/25 [==============================] - 0s - loss: 1.8592 - acc: 0.6400     
Epoch 353/1000
25/25 [==============================] - 0s - loss: 1.8603 - a

...

25/25 [==============================] - 0s - loss: 1.6153 - acc: 0.8400     
Epoch 605/1000
25/25 [==============================] - 0s - loss: 1.6127 - acc: 0.8400     
Epoch 606/1000
25/25 [==============================] - 0s - loss: 1.6128 - acc: 0.8800     
Epoch 607/1000
25/25 [==============================] - 0s - loss: 1.6105 - acc: 0.8400     
Epoch 608/1000
25/25 [==============================] - 0s - loss: 1.6134 - acc: 0.8000         
Epoch 609/1000
25/25 [==============================] - 0s - loss: 1.6094 - acc: 0.8000     
Epoch 610/1000
25/25 [==============================] - 0s - loss: 1.6085 - acc: 0.8800     
Epoch 611/1000
25/25 [==============================] - 0s - loss: 1.6076 - acc: 0.8800     
Epoch 612/1000
25/25 [==============================] - 0s - loss: 1.6066 - acc: 0.8400         
Epoch 613/1000
25/25 [==============================] - 0s - loss: 1.6062 - acc: 0.8400     
Epoch 614/1000
25/25 [==============================] - 0s - loss: 1.6071 - a

25/25 [==============================] - 0s - loss: 1.5513 - acc: 0.8800     
Epoch 692/1000
25/25 [==============================] - 0s - loss: 1.5507 - acc: 0.8400     
Epoch 693/1000
25/25 [==============================] - 0s - loss: 1.5506 - acc: 0.8800     
Epoch 694/1000
25/25 [==============================] - 0s - loss: 1.5518 - acc: 0.8800     
Epoch 695/1000
25/25 [==============================] - 0s - loss: 1.5517 - acc: 0.8800     
Epoch 696/1000
25/25 [==============================] - 0s - loss: 1.5513 - acc: 0.8800     
Epoch 697/1000
25/25 [==============================] - 0s - loss: 1.5478 - acc: 0.8400     
Epoch 698/1000
25/25 [==============================] - 0s - loss: 1.5497 - acc: 0.8400     
Epoch 699/1000
25/25 [==============================] - 0s - loss: 1.5475 - acc: 0.8400     
Epoch 700/1000
25/25 [==============================] - 0s - loss: 1.5453 - acc: 0.8400     
Epoch 701/1000
25/25 [==============================] - 0s - loss: 1.5460 - acc: 0.84

25/25 [==============================] - 0s - loss: 1.5007 - acc: 0.8800     
Epoch 779/1000
25/25 [==============================] - 0s - loss: 1.4991 - acc: 0.8800     
Epoch 780/1000
25/25 [==============================] - 0s - loss: 1.5002 - acc: 0.8400     
Epoch 781/1000
25/25 [==============================] - 0s - loss: 1.4981 - acc: 0.8800     
Epoch 782/1000
25/25 [==============================] - 0s - loss: 1.4977 - acc: 0.9200     
Epoch 783/1000
25/25 [==============================] - 0s - loss: 1.4966 - acc: 0.8400     
Epoch 784/1000
25/25 [==============================] - 0s - loss: 1.4956 - acc: 0.8800     
Epoch 785/1000
25/25 [==============================] - 0s - loss: 1.4955 - acc: 0.8800         
Epoch 786/1000
25/25 [==============================] - 0s - loss: 1.4963 - acc: 0.8800     
Epoch 787/1000
25/25 [==============================] - 0s - loss: 1.4925 - acc: 0.8800     
Epoch 788/1000
25/25 [==============================] - 0s - loss: 1.4943 - acc: 

25/25 [==============================] - 0s - loss: 1.4529 - acc: 0.8800     
Epoch 866/1000
25/25 [==============================] - 0s - loss: 1.4556 - acc: 0.8800         
Epoch 867/1000
25/25 [==============================] - 0s - loss: 1.4559 - acc: 0.9200     
Epoch 868/1000
25/25 [==============================] - 0s - loss: 1.4529 - acc: 0.8800     
Epoch 869/1000
25/25 [==============================] - 0s - loss: 1.4509 - acc: 0.9200     
Epoch 870/1000
25/25 [==============================] - 0s - loss: 1.4524 - acc: 0.8800     
Epoch 871/1000
25/25 [==============================] - 0s - loss: 1.4524 - acc: 0.8800         
Epoch 872/1000
25/25 [==============================] - 0s - loss: 1.4518 - acc: 0.9200     
Epoch 873/1000
25/25 [==============================] - 0s - loss: 1.4488 - acc: 0.9200     
Epoch 874/1000
25/25 [==============================] - 0s - loss: 1.4510 - acc: 0.9200     
Epoch 875/1000
25/25 [==============================] - 0s - loss: 1.4480 - a

25/25 [==============================] - 0s - loss: 1.4140 - acc: 0.9200     
Epoch 953/1000
25/25 [==============================] - 0s - loss: 1.4093 - acc: 0.9200     
Epoch 954/1000
25/25 [==============================] - 0s - loss: 1.4134 - acc: 0.9200         
Epoch 955/1000
25/25 [==============================] - 0s - loss: 1.4122 - acc: 0.9200     
Epoch 956/1000
25/25 [==============================] - 0s - loss: 1.4111 - acc: 0.9200     
Epoch 957/1000
25/25 [==============================] - 0s - loss: 1.4112 - acc: 0.9200     
Epoch 958/1000
25/25 [==============================] - 0s - loss: 1.4113 - acc: 0.8800     
Epoch 959/1000
25/25 [==============================] - 0s - loss: 1.4113 - acc: 0.8800     
Epoch 960/1000
25/25 [==============================] - 0s - loss: 1.4131 - acc: 0.8800     
Epoch 961/1000
25/25 [==============================] - 0s - loss: 1.4086 - acc: 0.9200     
Epoch 962/1000
25/25 [==============================] - 0s - loss: 1.4094 - acc: 

In [218]:
# summarize performance of the model
scores = model.evaluate(X, y, verbose=1)
print("Model Accuracy: {:0.2f} %".format(scores[1]*100))

25/25 [==============================] - 0s
Model Accuracy: 92.00 %


In [219]:
"""We can then re-run the training data through the network and generate predictions,
converting both the input and output pairs back into their original character format
to get a visual idea of how well the network learned the problem."""
# demonstrate some model predictions
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

['A'] -> B
['B'] -> C
['C'] -> D
['D'] -> E
['E'] -> F
['F'] -> G
['G'] -> H
['H'] -> I
['I'] -> J
['J'] -> K
['K'] -> L
['L'] -> M
['M'] -> N
['N'] -> O
['O'] -> P
['P'] -> Q
['Q'] -> R
['R'] -> S
['S'] -> T
['T'] -> U
['U'] -> V
['V'] -> W
['W'] -> Y
['X'] -> Z
['Y'] -> Z


In [280]:
for pattern in dataX:
    print("--------------------")
    print("pattern", pattern)
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    print(x)
    x = x / float(len(alphabet))
    print(x)
    prediction = model.predict(x, verbose=0)
    print("prediction", [np.round(x,3) for x in prediction] )
    index = numpy.argmax(prediction)
    print("index", index)
    #
    result = int_to_char[index]
    seq_in = [int_to_char[pattern[0]]]
    print(seq_in, "->", result)


--------------------
pattern [0]
[[[0]]]
[[[ 0.]]]
prediction [array([ 0.        ,  0.75800002,  0.212     ,  0.027     ,  0.003     ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ], dtype=float32)]
index 1
['A'] -> B
--------------------
pattern [1]
[[[1]]]
[[[ 0.03846154]]]
prediction [array([ 0.   ,  0.243,  0.491,  0.213,  0.044,  0.007,  0.001,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ], dtype=float32)]
index 2
['B'] -> C
--------------------
pattern [2]
[[[2]]]
[[[ 0.07692308]]]
prediction [array([ 0.        ,  0.013     ,  0.244     ,  0.42399999,  0.222     ,
        0.073     ,  0.02      ,  0.004   

0